In [23]:
import numpy as np
from worst_case_implementation import VecDBWorst
from hnsw import VecDBhnsw
import time
from dataclasses import dataclass
from typing import List
from vec_db import VecDB
# from inverted_file_index import VecDBIF
from math import ceil


In [1]:
import faiss
import numpy as np

# Create a dataset for demonstration
data = np.random.random((1000, 10)).astype(np.float32)

# Create an HNSW index
index = faiss.IndexHNSWFlat(10, 32)  # 10-dimensional vectors, 32 neighbors per point
index.add(data)

# Query for nearest neighbors
query = np.random.random((1, 10)).astype(np.float32)
distances, ids = index.search(query, k=5)

print("Nearest neighbor IDs:", ids)
print("Distances:", distances)


Nearest neighbor IDs: [[ 55 716 661 111 315]]
Distances: [[0.24350546 0.3126167  0.38367575 0.4391805  0.5038757 ]]


In [4]:
def _cal_score(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
    return cosine_similarity

In [7]:

QUERY_SEED_NUMBER = 10
DB_SEED_NUMBER = 50

rng = np.random.default_rng(QUERY_SEED_NUMBER)
query = rng.random((1, 70), dtype=np.float32)

num_records = 10**7*2
rng = np.random.default_rng(DB_SEED_NUMBER)
records_np = rng.random((num_records, 70), dtype=np.float32)

new_db = True

# calculate the score for each record and sort them, write them to a file
scores = []
for i in range(100000):
    score = _cal_score(query, records_np[i])
    scores.append((score, i))
    
scores.sort(key=lambda x: x[0])
with open("scores.txt", "w") as f:
    for score, index in scores:
        f.write(str(index) + "," + str(score) + "\n")


In [26]:
import faiss
# use faiss to build hnsw index
num_records = 10**7*2
rng = np.random.default_rng(DB_SEED_NUMBER)
records_np = rng.random((num_records, 70), dtype=np.float32)
index = faiss.IndexHNSWFlat(70, 16)
index.hnsw.efConstruction = 200
index.add(records_np)
faiss.write_index(index, "hnsw.index")

# generate the query
rng = np.random.default_rng(QUERY_SEED_NUMBER)
query = rng.random((1, 70), dtype=np.float32)

# use faiss to search
index = faiss.read_index("hnsw.index")
D, I = index.search(query, 100)
print(I)
print(D)

# compare the result with the result from brute force
scores = []
for i in range(100000):
    score = _cal_score(query, records_np[i])
    scores.append((score, i))
    
scores.sort(key=lambda x: x[0])
print(scores[:100])




In [ ]:
import nmslib
import numpy as np

# Create a dataset for demonstration
data = np.random.random((1000, 10)).astype(np.float32)

# Create an HNSW index
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)

# Query for nearest neighbors
query = np.random.random((1, 10)).astype(np.float32)
ids, distances = index.knnQuery(query, k=5)

print("Nearest neighbor IDs:", ids)
print("Distances:", distances)


In [8]:
x = 5
y = 6
z = x

# to let z be a reference to x instead of a copy of x, we can use the following
z = x
x = 7

print(x, y, z)

7 6 5


In [9]:
dictionary = {'a': 5, 'b': 2, 'c': 4}
sorted_dict = sorted(dictionary.items(), key=lambda x: x[1])
print(sorted_dict)
first_value = sorted_dict[0][1]
print(first_value)

[('b', 2), ('c', 4), ('a', 5)]
2


In [10]:
# for i in range(0,5):
#     print(i)

# now if we want to print in reverse order
for i in range(5,0,-1):
    print(i)

5
4
3
2
1


In [11]:
from typing import Dict, List, Annotated

l_of_dict: List[Dict[int, chr]] = [{1: 'a', 2: 'b'}, {3: 'c', 4: 'd'}, {5: 'e', 6: 'f', 7: 'g'}]

l_of_dict.append({})
print(len(l_of_dict))
print(l_of_dict[0].get(1))

4
a


In [12]:
l = [1, 2, 3, 4, 5]

for i in l[:3]:
    print(i)

1
2
3


In [13]:
for i in range (5):
    print(i)

0
1
2
3
4


In [14]:
l = []
for i in range(5):
    l.insert(0, {i: i})

print(l)

[{4: 4}, {3: 3}, {2: 2}, {1: 1}, {0: 0}]


In [15]:
l1= [1,2,3,4,5]
l2 = [6,7,8,9,10]
l3 = l1 + l2
print(l3)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [16]:
a = dict()
a[0] = 5
print(a)

{0: 5}


In [17]:
# now we will make a dictionary where the key is integer and value is a list of integers
b = dict()
# now we want to append only one value to the list
b[0] = b.get(0, []) + [1]
print(b)

b[0] = b.get(0, []) + [2]
print(b)

{0: [1]}
{0: [1, 2]}


In [18]:
# let's test the kmeans algorithm
from sklearn.cluster import KMeans
import numpy as np
class vector:
    def __init__(self, id, vect) -> None:
        self.id = id
        self.vect = vect
        self.centroid = None # the centroid that this vector belongs to


Vecs = []
Vecs.append(vector(0, [1, 2]))
Vecs.append(vector(1, [1, 4]))
Vecs.append(vector(2, [1, 0]))
Vecs.append(vector(3, [10, 2]))
Vecs.append(vector(4, [10, 4]))
Vecs.append(vector(5, [10, 0]))

# apply kmeans on the vectors
kmeans = KMeans(n_clusters=2, random_state=0).fit(np.array([v.vect for v in Vecs]))

print(kmeans.cluster_centers_)
print("------------------")
#  now we need to find the n closest centroids to the vector
n = 2
v = Vecs[0]
print(kmeans.predict([v.vect]))
print("------------------")

nearest_centroids = sorted(kmeans.cluster_centers_, key=lambda x: np.linalg.norm(x - v.vect), reverse= True)[:n]
print(nearest_centroids)
print("------------------")
# another way to do it is to use the kneighbors function



c:\Users\Rufai\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[[10.  2.]
 [ 1.  2.]]
------------------
[1]
------------------
[array([10.,  2.]), array([1., 2.])]
------------------


In [19]:
import heapq
heap = []

# heapify function converts a regular list to a heap
heapq.heapify(heap)
heapq.heappush(heap, (1, 'one'))
heapq.heappush(heap, (10, 'ten'))
heapq.heappush(heap, (5, 'five'))
heapq.heappush(heap, (2, 'two'))
print(heap)

# print only the first element of the heap
print(heap[0][0])


# now we will try to create another heap that is sorted descendingly
# we can do that by multiplying the key by -1
heap2 = []
heapq.heapify(heap2)
heapq.heappush(heap2, (-1, 'one'))
heapq.heappush(heap2, (-10, 'ten'))
heapq.heappush(heap2, (-5, 'five'))
heapq.heappush(heap2, (-2, 'two'))
print(heap2)




[(1, 'one'), (2, 'two'), (5, 'five'), (10, 'ten')]
1
[(-10, 'ten'), (-2, 'two'), (-5, 'five'), (-1, 'one')]


In [20]:
# Nothing changed
!conda update sklearn

# Error cannot find pip3
!python3 -m pip3 install --upgrade sklearn

# From the logs is installing 0.22
!pip install sklearn --upgrade


!conda install scikit-learn -y

# Stuck forever at: Solving environment
!conda config --append channels conda-forge
!conda install scikit-learn=0.23.1

'conda' is not recognized as an internal or external command,
operable program or batch file.
C:\msys64\mingw64\bin\python3.exe: No module named pip3
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Python311\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'
'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.


In [21]:
# uinstall pip threadpoolctl
# uninstall sklearn 
# install sklearn

# pip freeze | grep scikit-learn  --> google colab

In [22]:
import pickle

# now we will try to load data from a pickle file
data = pickle.load(open('10k/centroids.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: '10k/centroids.pkl'

In [ ]:
import pickle

person1 = [1, 2, 3, 4, 5]
preson2 = [6, 7, 8, 9, 10]

pickleFile = open("person1.pkl","wb")
pickle.dump(person1, pickleFile) # put
pickle.dump(preson2, pickleFile) # put



pickleFile = open("person1.pkl","rb")

# looad from the file until we reach the end of the file
while True:
    try:
        print(pickle.load(pickleFile))
    except EOFError:
        break

# print(persona)
# print(personb)

[1, 2, 3, 4, 5]
[6, 7, 8, 9, 10]


In [ ]:
matrix = [[1,2,3],[4,5,6],[7,8,9]]
# let's remove the first element of each row without using a for loop
matrix = [row[1:] for row in matrix]
print(matrix)

[[2, 3], [5, 6], [8, 9]]


In [ ]:
v = [1] + [2,3,4]
print(v)

[1, 2, 3, 4]


In [ ]:
data_list = [
    {'example_key': 'example_value_1', 'numbers': [1, 2, 3, 4]},
    {'example_key': 'example_value_2', 'numbers': [5, 6, 7, 8]},
    # Add more dictionaries as needed
]

# Specify the file path
file_path = 'example.pkl'

# Open the file in binary write mode ('wb')
with open(file_path, 'wb') as file:
    # Use a for loop to iterate through the list and dump each dictionary
    for data in data_list:
        pickle.dump(data, file)

# Open the file in binary read mode ('rb')
with open(file_path, 'rb') as file:
    # Use a for loop to iterate through the list and load each dictionary
    for i in range(len(data_list)):
        data_list[i] = pickle.load(file)

# Print the loaded data
print(data_list)


[{'example_key': 'example_value_1', 'numbers': [1, 2, 3, 4]}, {'example_key': 'example_value_2', 'numbers': [5, 6, 7, 8]}]


In [21]:
import shutil
import os

def copy_files(src_folder, dest_folder):
    # Make sure the source folder exists
    if not os.path.exists(src_folder):
        print(f"Source folder '{src_folder}' does not exist.")
        return

    # Make sure the destination folder exists, create it if necessary
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # Get a list of all files in the source folder
    files = os.listdir(src_folder)

    # Copy each file from the source folder to the destination folder
    for file in files:
        src_path = os.path.join(src_folder, file)
        dest_path = os.path.join(dest_folder, file)
        shutil.copy(src_path, dest_path)
        print(f"File '{file}' copied to '{dest_folder}'.")

# Example usage:
source_folder = 'src'
destination_folder = 'dest'

copy_files(source_folder, destination_folder)




File 'hello.txt' copied to 'dest'.


In [ ]:
# let's append in the file in the dest folder
file = open(f'{destination_folder}/hello.txt', 'a')
file.write('doaa magdy')

10

In [16]:
import numpy as np
rng = np.random.default_rng(50)
query = rng.random((1, 70), dtype=np.float32)
print(query)
query.reshape(1, -1)
query = np.array(query).tolist()
print(query)


query = [float(e[0]) for e in query]
print(query)

[[0.7813332  0.78742266 0.7145687  0.8336693  0.8237809  0.54790443
  0.24254471 0.97344905 0.8289076  0.23683351 0.27866644 0.6469233
  0.67119896 0.06502604 0.9596958  0.5555776  0.01293612 0.40235746
  0.6543596  0.26596576 0.6883337  0.9866823  0.6262581  0.39731765
  0.18344492 0.32845616 0.6601709  0.12515694 0.99986154 0.11145777
  0.29785854 0.98457336 0.55596834 0.5789395  0.45138943 0.8920926
  0.53687006 0.01508015 0.78798324 0.59969866 0.1192258  0.4202239
  0.54847825 0.37373084 0.9542997  0.70650905 0.27028292 0.06140804
  0.542406   0.23580968 0.91909045 0.24576241 0.6833456  0.44861054
  0.22162598 0.3316726  0.462358   0.7717461  0.15772003 0.47309273
  0.58490956 0.7804164  0.4965189  0.12486297 0.46206492 0.57882965
  0.24902534 0.70675725 0.6981578  0.6835448 ]]
[[0.7813332080841064, 0.7874226570129395, 0.7145686745643616, 0.8336693048477173, 0.82378089427948, 0.5479044318199158, 0.24254471063613892, 0.9734490513801575, 0.8289076089859009, 0.23683351278305054, 0.278

In [3]:
A = "DOAA"
B = "DOAA"
print(B == A)


True


In [8]:
import heapq

heap = []
heapq.heapify(heap)
heapq.heappush(heap, (1, 'one'))
heapq.heappush(heap, (10, 'ten'))
heapq.heappush(heap, (5, 'five'))
heapq.heappush(heap, (0, "zero"))

h2 = []
heapq.heapify(h2)
h2 += heap[0:3]

print(h2)
print(len(heap))

[(0, 'zero'), (1, 'one'), (5, 'five')]
4


In [2]:
# use double ended priority queue
import collections

q = collections.deque()

q.append((5, "five"))
q.append((1, "one"))
q.append((10, "ten"))

print(q)

print(q.popleft())
print(q)

# but the prev q is not sorted, to make a priority queue we can use the following


deque([(5, 'five'), (1, 'one'), (10, 'ten')])
(5, 'five')
deque([(1, 'one'), (10, 'ten')])


In [19]:
import numpy as np

vectors = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
normalized_vectors = vectors / np.linalg.norm(vectors, axis=1)[:, np.newaxis]

print("Original Vectors:\n", vectors)
print("Normalized Vectors:\n", normalized_vectors)

Original Vectors:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Normalized Vectors:
 [[0.26726124 0.53452248 0.80178373]
 [0.45584231 0.56980288 0.68376346]
 [0.50257071 0.57436653 0.64616234]]


In [15]:
import heapq
heap = []
heapq.heapify(heap)
heapq.heappush(heap, (1, 'one'))
heapq.heappush(heap, (10, 'ten'))
heapq.heappush(heap, (5, 'five'))

# remove heap[-1] from the heap
heap = heap[:-2]

print(len(heap))

for h in range(3):
    print(heap[-1])

1
(1, 'one')
(1, 'one')
(1, 'one')
